# Initialization

In [3]:
!nvidia-smi

Fri Mar 19 22:16:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Install RAPIDS
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!bash rapidsai-csp-utils/colab/rapids-colab.sh stable

import sys, os

dist_package_index = sys.path.index('/usr/local/lib/python3.7/dist-packages')
sys.path = sys.path[:dist_package_index] + ['/usr/local/lib/python3.7/site-packages'] + sys.path[dist_package_index:]
sys.path
exec(open('rapidsai-csp-utils/colab/update_modules.py').read(), globals())

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 205 (delta 16), reused 3 (delta 0), pack-reused 171
Receiving objects: 100% (205/205), 62.72 KiB | 1.61 MiB/s, done.
Resolving deltas: 100% (78/78), done.
PLEASE READ
********************************************************************************************************
Changes:
1. IMPORTANT SCRIPT CHANGES: Colab has updated to Python 3.7, and now runs our STABLE and NIGHTLY versions (0.18 and 0.19)!  PLEASE update your older install script code as follows:
	!bash rapidsai-csp-utils/colab/rapids-colab.sh 0.18

	import sys, os

	dist_package_index = sys.path.index('/usr/local/lib/python3.7/dist-packages')
	sys.path = sys.path[:dist_package_index] + ['/usr/local/lib/python3.7/site-packages'] + sys.path[dist_package_index:]
	sys.path
	exec(open('rapidsai-csp-utils/colab/update_modules.py').read(), globals(

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Download Kaggle Data

In [ ]:
!pip install -q kaggle
from google.colab import files
files.upload()

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c shopee-product-matching
!unzip -q shopee-product-matching.zip -d ./drive/MyDrive/data/shopee
!rm shopee-product-matching.zip

## Import Packages

In [5]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import EfficientNetB3
from cuml.neighbors import NearestNeighbors
from cuml.feature_extraction.text import TfidfVectorizer
import gc
import pandas as pd
import numpy as np
import cudf, cuml, cupy
import pickle

In [6]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
SEED = 100
BATCH_SIZE =32
CHUNK_SIZE = 2048
IMAGE_HEIGHT = 224
IMAGE_WIDTH = 224
LIMIT = 2.0
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs:", len(physical_devices))

if physical_devices:
  tf.config.experimental.set_virtual_device_configuration(
      physical_devices[0],
      [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*LIMIT)])
  print('TensorFlow usage is restricted to max %iGB GPU RAM'%LIMIT)

Num GPUs: 1
TensorFlow usage is restricted to max 2GB GPU RAM


# Process Data

In [7]:
def preprocess_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [IMAGE_WIDTH, IMAGE_HEIGHT])
#     image /= 255  # normalize to [0,1] rangeI'm not a 
    return image

def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    return preprocess_image(image)

def augmentation(ds):
    data_augmentation = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.3),
        layers.experimental.preprocessing.RandomTranslation(
            height_factor=(-0.2, 0.2), width_factor=(-0.2, 0.2)),
        layers.experimental.preprocessing.RandomZoom(0.2, 0.2),
    ])
    
    # Batch all datasets
    ds = ds.batch(BATCH_SIZE)

    # Use data augmentation only on the training set
    ds = ds.map(lambda x: data_augmentation(x))

    # Prefecting on all datasets
    return ds.prefetch(1)

def prepare_data(df, augment=False):
    # Load images
    path_ds = tf.data.Dataset.from_tensor_slices(df['image_paths'])
    image_ds = path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)

    if augment:
        ds = augmentation(image_ds)
    else:
        ds = image_ds.batch(BATCH_SIZE).prefetch(1)
    
    return ds

In [8]:
load_dir = './drive/MyDrive/data/shopee'

# Load and process images
df_train = pd.read_csv(load_dir + '/train.csv')
df_train['image_paths'] = load_dir + '/train_images/' + df_train['image'] 

# Duplicate train set for runtime testing 
# df_train = pd.concat([df_train, df_train], ignore_index=True)
train_ds = prepare_data(df_train, augment=False)

# Embeddings

In [9]:
def find_similar_items(embeddings, threshold, fine_tune=False):
  """
  Using Nearest Neighbors to figure out similar items
  """
  knn = NearestNeighbors(n_neighbors=50)
  knn.fit(embeddings)

  num_chunk = round(embeddings.shape[0] / CHUNK_SIZE)
  item_index = []
  for i in range(num_chunk+1):
      start_idx = i * CHUNK_SIZE                  
      end_idx = min((i + 1) * CHUNK_SIZE, embeddings.shape[0])
      if not fine_tune:
        print('Chunk', start_idx, 'to', end_idx) 

      dist, idx = knn.kneighbors(embeddings[start_idx:end_idx, :])
      counts = (dist < threshold).sum(axis=1)
      chunk_index = [idx[i, :counts[i]].tolist() for i in range(end_idx - start_idx)]
      item_index += chunk_index

  return item_index

In [ ]:
# Image embedding
effnet = EfficientNetB3(weights='imagenet', include_top=False, pooling='avg', input_shape=None)
embeddings_image = effnet.predict(train_ds, verbose=1)

# num_chunk = round(len(df_train) / CHUNK_SIZE)
# embeddings_image = []
# for i in range(num_chunk+1):
#     start_idx = i * CHUNK_SIZE                  
#     end_idx = min((i + 1) * CHUNK_SIZE, len(df_train))
#     print('Chunk', start_idx, 'to', end_idx) 
    
#     train_ds = prepare_data(df_train[start_idx:end_idx], augment=False)
#     embeddings_image.append(effnet.predict(train_ds, verbose=1))

# embeddings_image =np.concatenate(embeddings_image)

del effnet
_ = gc.collect()

In [12]:
with open(load_dir + '/embeddings_image_B3.pickle', 'rb') as f:
  embeddings_image = pickle.load(f)

In [ ]:
# Similiar images
image_index = find_similar_items(embeddings_image, 6.8)

Chunk 0 to 2048
Chunk 2048 to 4096
Chunk 4096 to 6144
Chunk 6144 to 8192
Chunk 8192 to 10240
Chunk 10240 to 12288
Chunk 12288 to 14336
Chunk 14336 to 16384
Chunk 16384 to 18432
Chunk 18432 to 20480
Chunk 20480 to 22528
Chunk 22528 to 24576
Chunk 24576 to 26624
Chunk 26624 to 28672
Chunk 28672 to 30720
Chunk 30720 to 32768
Chunk 32768 to 34250


In [ ]:
# Text embedding
cudf_train = cudf.DataFrame(df_train)
sentences = cudf_train.title
vectorizer = TfidfVectorizer(binary=True, max_features=15000)
embeddings_text = vectorizer.fit_transform(sentences).toarray()

del sentences, vectorizer
_ = gc.collect()

In [ ]:
# Similar texts
text_index = find_similar_items(embeddings_text, 0.8)

# Evaluation

In [ ]:
def row_wise_f1_score(y_true, y_pred):

    y_true = y_true.apply(lambda x: set(x))
    y_pred = y_pred.apply(lambda x: set(x.split()))

    tp = cupy.array([len(x[0] & x[1]) for x in zip(y_true, y_pred)])
    fp = y_pred.apply(lambda x: len(x)).values - tp
    fn = y_true.apply(lambda x: len(x)).values - tp

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * ((precision * recall) / (precision + recall))
    return f1

def parameter_tuning(df_train, text_index, embeddings, threshold):
    print('threshold:', round(threshold, 2))
    image_index = search_similar_image(embeddings, threshold)
    df_train['matches'] = [
        ' '.join(
            set(df_train['posting_id'][text].tolist() +
                df_train['posting_id'][image].tolist()))
        for text, image in zip(text_index, image_index)
    ]
    df_train['f1'] = row_wise_f1_score(df_train['target'], df_train['matches'])
    f1_score = df_train['f1'].mean()
    
    print('f1-score:', round(f1_score, 3))
    print('\n')

In [ ]:
# Ground truth
tmp = cudf_train.groupby('label_group').posting_id.agg('unique').to_dict()
cudf_train['target'] = cudf_train.label_group.map(tmp)

cudf_train['matches'] = [
    ' '.join(
        set(cudf_train['posting_id'][text].tolist() +
            cudf_train['posting_id'][image].tolist()))
    for text, image in zip(text_index, image_index)
]

cudf_train['f1'] = row_wise_f1_score(cudf_train['target'], cudf_train['matches'])
cudf_train['f1'].mean()